In [17]:
import re
import ast
import numpy as np
import math
from functools import cache, cmp_to_key
from itertools import product

def read_file_to_string(file_path):
    with open(file_path,'r') as f:
        return f.read()
    
day_22_string = read_file_to_string('day_22_input.txt')
day_22_test = read_file_to_string('day_22_test.txt')
#day_20_test_2 = read_file_to_string('day_20_test_2.txt')
# day_20_test_3 = read_file_to_string('day_20_test_3.txt')
# day_20_test_4 = read_file_to_string('day_20_test_4.txt')
# day_20_test_5 = read_file_to_string('day_20_test_5.txt')

In [8]:
def parse_string(string):
    day_22=string.split('\n')
    day_22=[tuple([tuple([int(coord) for coord in coords.split(',')]) for coords in row.split('~')]) for row in day_22]

    return day_22

In [212]:
input_bricks = parse_string(day_22_string)
test_bricks = parse_string(day_22_test)

In [12]:
def third_brick_compare(brick1,brick2):
    if brick1[0][2] < brick2[0][2]:
        return -1
    elif brick1[0][2] > brick2[0][2]:
        return 1
    else:
        return 0

In [269]:
def elongate_brick(brick):

    for i in range(3):
        if brick[1][i] - brick[0][i] > 0:

            if i == 0:
                return [(j,brick[0][1],brick[0][2]) for j in range(brick[0][i], brick[1][i]+1)]

            elif i == 1:
                return [(brick[0][0],j,brick[0][2]) for j in range(brick[0][i], brick[1][i]+1)]

            else:
                return [(brick[0][0],brick[0][1],j) for j in range(brick[0][i], brick[1][i]+1)]
            
    return [brick[0]]

In [87]:
def is_brick_under(brick_layer1,brick2):

    #check if brick1 is UNDER brick2

    #first - make each brick a list of all the coords it covers.
    elon_brick1_xy = []
    for brick in brick_layer1:
        elon_brick = elongate_brick(brick)
        elon_brick1_xy += [(point[0],point[1]) for point in elon_brick]
    
    elon_brick2 = elongate_brick(brick2)
    elon_brick2_xy = [(point[0],point[1]) for point in elon_brick2]
    # print(elon_brick1_xy)
    # print(elon_brick2_xy)

    #first check: is z axis equal or greater?

    # if brick1[0][3] > brick2[0][3]:
    #     return False
    
    for point in elon_brick2_xy:
        #print(point)
        if point in elon_brick1_xy:
            return True
    
    return False

In [88]:
is_brick_under([((1, 0, 1), (1, 2, 1))],((0, 0, 2), (2, 0, 2)))

True

In [261]:
#this is causing a huge bleeding issue
def remove_brick_gaps(brick_list):
    #make sure list is sorted!!

    #print(brick_list[1:])
    difference = 0
    for i, brick in enumerate(brick_list):
        #print(brick_list[i][0][2] - brick_list[i-1][0][2])
        if i != 0:
            if brick_list[i][0][2] - brick_list[i-1][0][2] >= 2:
                #print(i, brick_list[i], brick_list[i-1])
                #the issue here is that fall count is going to fuck up as it'll get exponentially bigger due to the fact we've not looked at the gap. We probably don't need fall count lol
                #print("hello")
                difference = brick_list[i][0][2] - brick_list[i-1][0][2] - 1
                #if i in [183,234,235,672,1253,1294]:
                    # print(brick_list[i][0][2],brick_list[i-1][0][2],difference)
                    # print(i, brick_list[i], brick_list[i-1])
        brick_list[i] = ((brick_list[i][0][0],brick_list[i][0][1],brick_list[i][0][2]-difference),(brick_list[i][1][0],brick_list[i][1][1],brick_list[i][1][2]-difference))

    return brick_list

In [308]:
def brick_height(brick):
    return brick[1][2]-brick[0][2]

In [413]:
def draw_bricks(brick_list):
    #realise this could make things a lot easier in the poker hands puzzle
    brick_list.sort(key=cmp_to_key(third_brick_compare))

    height_map = {
        [i for i in range(brick_list[-1][0][-1]+1)][j]: [] for j in range(brick_list[-1][0][-1]+1)
    }

    if 0 in height_map.keys():
        del height_map[0]

    for brick in brick_list:
        brick_start_height = brick[0][-1]
        height_map[brick_start_height].append(brick)

    for k in list(height_map.keys())[::-1][:-1]:

        falling_brick = []
        
        layer = height_map[k-1]
        layer_addition = []
        for j in range(k-1):
            if j in height_map.keys():
                for brick2 in height_map[j]:
                    if brick_height(brick2) == k-1-j:
                        layer_addition.append(brick2)

        for brick in height_map[k]:
            if not is_brick_under(layer+layer_addition,brick):
                falling_brick.append(brick)
        
        for brick in falling_brick:
            height_map[k-1].append(brick)
            height_map[k].remove(brick)

    return height_map

In [371]:
test_height_map = draw_bricks(test_bricks)

In [379]:
def can_I_disintegrate(brick, brick_layer, height_map):

    #idea - take the layer but remove the brick under. If is_brick_under returns True for all bricks above, we can disintegrate the brick
    #new_height_map = height_map.copy()
    
    index = height_map[brick_layer].index(brick)

    h = brick_height(brick)
    layer_addition = []
    for j in range(brick_layer+h):
        if j in height_map.keys():
            for brick2 in height_map[j]:
                if brick_height(brick2) == brick_layer+h-j:
                    layer_addition.append(brick2)
        

    if brick_height(brick) == 0:
        test_layer = height_map[brick_layer][:index] + height_map[brick_layer][index+1:] + layer_addition
    else:
        #print(brick, layer_addition)
        layer_addition.remove(brick)
        test_layer = height_map[brick_layer+brick_height(brick)] + height_map[brick_layer+brick_height(brick)] + layer_addition
    
    if brick_layer+brick_height(brick)+1 in height_map.keys():
        for brick in height_map[brick_layer+brick_height(brick)+1]:
            if not is_brick_under(test_layer,brick):
                return False
       
    return True

In [684]:
def part_one(height_map):
    new_height_map = height_map.copy()

    disintegrate_list=[]
    for k,v in list(height_map.items())[:-1]:

        for brick in v:
            if can_I_disintegrate(brick, k, new_height_map):
                disintegrate_list.append(brick)

    disintegrate_list += height_map[len(height_map)-1]

    return disintegrate_list

In [435]:
def check_again(height_map):
    
    for k in list(height_map.keys())[::-1][:-1]:

        falling_brick = []

        #generate layer additions
        layer = height_map[k-1]
        layer_addition = []
        for j in range(k-1):
            if j in height_map.keys():
                for brick2 in height_map[j]:
                    if brick_height(brick2) == k-1-j:
                        layer_addition.append(brick2)
        # if layer_addition!=[]:
        #     print(layer_addition)

        for brick in height_map[k]:
            if not is_brick_under(layer+layer_addition,brick):
                falling_brick.append(brick)
        
        for brick in falling_brick:
            height_map[k-1].append(brick)
            height_map[k].remove(brick)

    return height_map

def get_the_map_right(height_map):
    
    new_height_map = check_again(height_map)
    old_height_map = height_map.copy()

    while new_height_map != old_height_map:

        old_height_map = new_height_map.copy()
        new_height_map = check_again(new_height_map)

    new_height_map = {[i for i in range(len(new_height_map))][j]:list(new_height_map.values())[j] for j in range(len(new_height_map))}

    return new_height_map

In [527]:
test_height_map = draw_bricks(test_bricks)
for i in range(6):
    test_height_map= get_the_map_right(test_height_map)
part_one(test_height_map)

5

In [489]:
test_height_map= get_the_map_right(test_height_map)
print(test_height_map)
part_one(test_height_map)

{0: [((1, 0, 1), (1, 2, 1))], 1: [((0, 0, 2), (2, 0, 2)), ((0, 2, 3), (2, 2, 3))], 2: [((0, 0, 4), (0, 2, 4)), ((2, 0, 5), (2, 2, 5))], 3: [((0, 1, 6), (2, 1, 6))], 4: [((1, 1, 7), (1, 1, 8))], 5: [], 6: []}


5

In [441]:
#print(input_bricks[184:])
input_height_map = draw_bricks(input_bricks)
for i in range(291):
    input_height_map= get_the_map_right(input_height_map)

#for some reason get_the_map_right doesn't sort the index

#x should be in that bleeding list!!
part_one(input_height_map)

242

In [487]:
for i in range(1000):
    input_height_map= get_the_map_right(input_height_map)
    print(part_one(input_height_map))

508
468
462
491
501
490
480
500
505
528
491
474
486
477
460
470
463
473
495
473
476
480
482
489
480
477
488
485
438
427
449
450
445
438
432
407
436
434
435
447
444
427
432
439
437
434
437
438
436
428
419
406
408
416
425
429
428
429
429
420
416
415
418
417
418
416
414
416
413
411
409
407
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405
405


KeyboardInterrupt: 

## Part 2

In [506]:
height_map_2 = test_height_map.copy()

In [507]:
test_height_map

{0: [((1, 0, 1), (1, 2, 1))],
 1: [((0, 0, 2), (2, 0, 2)), ((0, 2, 3), (2, 2, 3))],
 2: [((0, 0, 4), (0, 2, 4)), ((2, 0, 5), (2, 2, 5))],
 3: [((0, 1, 6), (2, 1, 6))],
 4: [((1, 1, 7), (1, 1, 8))],
 5: [],
 6: []}

In [508]:
del height_map_2[5]

In [509]:
test_height_map

{0: [((1, 0, 1), (1, 2, 1))],
 1: [((0, 0, 2), (2, 0, 2)), ((0, 2, 3), (2, 2, 3))],
 2: [((0, 0, 4), (0, 2, 4)), ((2, 0, 5), (2, 2, 5))],
 3: [((0, 1, 6), (2, 1, 6))],
 4: [((1, 1, 7), (1, 1, 8))],
 5: [],
 6: []}

In [510]:
height_map_2

{0: [((1, 0, 1), (1, 2, 1))],
 1: [((0, 0, 2), (2, 0, 2)), ((0, 2, 3), (2, 2, 3))],
 2: [((0, 0, 4), (0, 2, 4)), ((2, 0, 5), (2, 2, 5))],
 3: [((0, 1, 6), (2, 1, 6))],
 4: [((1, 1, 7), (1, 1, 8))],
 6: []}

In [593]:
def can_I_disintegrate_multiple(bricks, brick_layer, height_map):

    #print(bricks)

    #idea - take the layer but remove the brick under. If is_brick_under returns True for all bricks above, we can disintegrate the brick
    #new_height_map = height_map.copy()

    brick_height_map = {}

    for brick in bricks:
        #print(brick)
        if brick_height(brick) not in brick_height_map:
            brick_height_map[brick_height(brick)] = [brick]
        else:
            brick_height_map[brick_height(brick)].append(brick)

    condition = True
    for height,bricks in brick_height_map.items():

        if len(bricks) == 1:
            condition = condition and can_I_disintegrate(bricks[0],brick_layer,height_map)
            

        else:
            h = height
            layer_addition = []
            for j in range(brick_layer+h):
                if j in height_map.keys():
                    for brick2 in height_map[j]:
                        if brick_height(brick2) == brick_layer+h-j:
                            layer_addition.append(brick2)
            
            if height == 0:
                test_layer = [brick for brick in height_map[brick_layer] if brick not in bricks] + layer_addition
            
                    

                
            if height != 0:
                    #print(brick, layer_addition)
                for brick in bricks:
                    layer_addition.remove(brick)
                test_layer = height_map[brick_layer+height] + height_map[brick_layer+height] + layer_addition
            
            if brick_layer+height+1 in height_map.keys():
                for brick in height_map[brick_layer+height+1]:
                    if not is_brick_under(test_layer,brick):
                        condition = False
            condition = condition and True
    return condition

In [663]:
#need to get the sum of this for every brick
#@cache
def how_many_fall(height_map,bricks,brick_layer):

    if bricks == []:
        return 0

    #if here from recurrence they should be the same layer
    height_map_2 = height_map.copy()

    if can_I_disintegrate_multiple(bricks, brick_layer,height_map_2):
        return 0

    else:

        brick_height_map = {}

        for brick in bricks:
            #print(brick)
            if brick_height(brick) not in brick_height_map:
                brick_height_map[brick_height(brick)] = [brick]
            else:
                brick_height_map[brick_height(brick)].append(brick)
        
        final_count = 0

        for height,bricks in brick_height_map.items():

            h = brick_height(brick)
            layer_addition = []
            for j in range(brick_layer+h):
                if j in height_map.keys():
                    for brick2 in height_map[j]:
                        if brick_height(brick2) == brick_layer+h-j:
                            layer_addition.append(brick2)

            if brick_height(brick) == 0:
                test_layer = [brick for brick in height_map[brick_layer] if brick not in bricks] + layer_addition

            else:
                #print(brick, layer_addition)
                layer_addition.remove(brick)
                test_layer = height_map[brick_layer+brick_height(brick)] + height_map[brick_layer+brick_height(brick)] + layer_addition

            if len(bricks) == 1:
                if can_I_disintegrate(bricks[0],brick_layer,height_map):
                    return 0
                else:
                    bricks_that_fall = []
                    for brick2 in height_map[brick_layer+height+1]:
                        if not is_brick_under(test_layer,brick2):
                            bricks_that_fall.append(brick2)
                    final_count += 1+how_many_fall(height_map_2,bricks_that_fall,brick_layer+height+1)

            else:
               if can_I_disintegrate_multiple(bricks,brick_layer,height_map):
                   return 0
               else:
                   bricks_that_fall = []
                   for brick2 in height_map[brick_layer+height+1]:
                        if not is_brick_under(test_layer,brick2):
                            bricks_that_fall.append(brick2)
                   final_count += len(bricks)+how_many_fall(height_map_2,bricks_that_fall,brick_layer+height+1)

        return final_count

In [692]:
def part_2(height_map,disintegrate_list):

    sum_ = 0
    for k,v in height_map.items():

        for brick in v:
            if brick not in disintegrate_list:
                sum_ += how_many_fall(height_map,[brick],k)
                print(brick, sum_)

    return sum_

In [690]:
new_input_height_map = input_height_map.copy()

In [698]:
def how_many_fall_2(height_map, brick, brick_layer):

    height_map[brick_layer].remove(brick)

    bricks_fallen = 0

    for k in list(height_map.keys())[1:]:

        falling_brick = []
        
        layer = height_map[k-1]
        layer_addition = []
        for j in range(k-1):
            if j in height_map.keys():
                for brick2 in height_map[j]:
                    if brick_height(brick2) == k-1-j:
                        layer_addition.append(brick2)

        for brick in height_map[k]:
            if not is_brick_under(layer+layer_addition,brick):
                falling_brick.append(brick)

        bricks_fallen += len(falling_brick)
        
        for brick in falling_brick:
            height_map[k-1].append(brick)
            height_map[k].remove(brick)


    return bricks_fallen

In [699]:
def part_2_2(height_map,disintegrate_list):

    sum_ = 0
    for k,v in height_map.items():

        for brick in v:
            if brick not in disintegrate_list:
                sum_ += how_many_fall_2(height_map,brick,k)
                print(brick, sum_)

    return sum_

In [701]:
part_one(test_height_map)

[((0, 0, 2), (2, 0, 2)),
 ((0, 2, 3), (2, 2, 3)),
 ((0, 0, 4), (0, 2, 4)),
 ((2, 0, 5), (2, 2, 5)),
 ((1, 1, 7), (1, 1, 8))]

In [700]:
part_2_2(test_height_map,part_one(test_height_map))

((0, 1, 6), (2, 1, 6)) 5


5

In [688]:
def part_2(height_map,disintegrate_list):

    sum_ = 0
    for k,v in height_map.items():

        for brick in v:
            if brick not in disintegrate_list:
                sum_ += how_many_fall(height_map,[brick],k)
                print(brick, sum_)

    return sum_

In [665]:
test_height_map

{0: [((1, 0, 1), (1, 2, 1))],
 1: [((0, 0, 2), (2, 0, 2)), ((0, 2, 3), (2, 2, 3))],
 2: [((0, 0, 4), (0, 2, 4)), ((2, 0, 5), (2, 2, 5))],
 3: [((0, 1, 6), (2, 1, 6))],
 4: [((1, 1, 7), (1, 1, 8))],
 5: [],
 6: []}

In [666]:
test_height_map = draw_bricks(test_bricks)
for i in range(6):
    test_height_map= get_the_map_right(test_height_map)
part_one(test_height_map)

5

In [667]:
#far too big!! double what it should be for (1,0,1), (1,2,1)
part_2(test_height_map)

((1, 0, 1), (1, 2, 1)) 6
((0, 0, 2), (2, 0, 2)) 6
((0, 2, 3), (2, 2, 3)) 6
((0, 0, 4), (0, 2, 4)) 6
((2, 0, 5), (2, 2, 5)) 6
((0, 1, 6), (2, 1, 6)) 7
((1, 1, 7), (1, 1, 8)) 7


7

In [668]:
#because it's deleting everything we don't actually get the correct value

In [669]:
#need to find a way to account for multiple in a layer being removed. Best way to do this is to rewrite 

In [655]:
input_height_map = draw_bricks(input_bricks)
for i in range(291):
    input_height_map= get_the_map_right(input_height_map)

part_one(input_height_map)

405

In [685]:
can_dis = part_one(input_height_map)

In [687]:
can_dis

[((4, 0, 1), (5, 0, 1)),
 ((4, 1, 1), (6, 1, 1)),
 ((3, 1, 1), (3, 1, 3)),
 ((9, 6, 1), (9, 6, 2)),
 ((8, 2, 1), (8, 4, 1)),
 ((9, 2, 1), (9, 3, 1)),
 ((7, 2, 2), (7, 4, 2)),
 ((9, 0, 3), (9, 0, 4)),
 ((1, 6, 3), (1, 6, 4)),
 ((1, 9, 3), (1, 9, 4)),
 ((9, 1, 5), (9, 1, 5)),
 ((5, 8, 4), (6, 8, 4)),
 ((8, 7, 5), (8, 9, 5)),
 ((7, 6, 6), (8, 6, 6)),
 ((5, 3, 4), (5, 5, 4)),
 ((3, 2, 5), (3, 2, 6)),
 ((1, 8, 7), (1, 8, 8)),
 ((0, 6, 8), (0, 6, 10)),
 ((3, 4, 7), (3, 4, 8)),
 ((9, 2, 6), (9, 4, 6)),
 ((7, 1, 8), (9, 1, 8)),
 ((1, 2, 8), (1, 3, 8)),
 ((6, 7, 7), (8, 7, 7)),
 ((4, 0, 10), (5, 0, 10)),
 ((0, 4, 11), (2, 4, 11)),
 ((2, 5, 11), (2, 5, 12)),
 ((3, 5, 17), (5, 5, 17)),
 ((8, 4, 13), (8, 6, 13)),
 ((2, 0, 13), (3, 0, 13)),
 ((4, 6, 18), (5, 6, 18)),
 ((7, 8, 15), (9, 8, 15)),
 ((7, 7, 15), (9, 7, 15)),
 ((5, 1, 15), (8, 1, 15)),
 ((6, 3, 17), (7, 3, 17)),
 ((6, 2, 17), (7, 2, 17)),
 ((3, 6, 20), (3, 7, 20)),
 ((4, 2, 18), (4, 3, 18)),
 ((5, 1, 19), (5, 4, 19)),
 ((9, 7, 24), (9, 9

In [689]:
part_2(input_height_map, can_dis)

((1, 9, 1), (3, 9, 1)) 1
((2, 6, 1), (2, 6, 3)) 5
((6, 4, 1), (6, 4, 3)) 66
((7, 6, 1), (7, 8, 1)) 68
((9, 4, 2), (9, 4, 3)) 70
((0, 5, 2), (2, 5, 2)) 72
((2, 0, 2), (2, 0, 5)) 84
((1, 3, 2), (1, 3, 5)) 85
((7, 1, 2), (9, 1, 2)) 86
((4, 2, 2), (4, 4, 2)) 89
((0, 8, 2), (2, 8, 2)) 91
((6, 8, 3), (6, 9, 3)) 92
((2, 4, 3), (3, 4, 3)) 94
((8, 9, 3), (9, 9, 3)) 95
((7, 5, 3), (7, 7, 3)) 96
((5, 1, 3), (7, 1, 3)) 98
((2, 3, 3), (5, 3, 3)) 99
((3, 2, 4), (5, 2, 4)) 100
((0, 8, 4), (3, 8, 4)) 101
((0, 5, 5), (0, 7, 5)) 102
((3, 4, 5), (3, 7, 5)) 103
((9, 3, 5), (9, 4, 5)) 104
((6, 1, 6), (7, 1, 6)) 105
((2, 7, 6), (4, 7, 6)) 113
((6, 3, 4), (6, 6, 4)) 173
((2, 4, 5), (2, 6, 5)) 176
((4, 6, 9), (4, 9, 9)) 183
((6, 6, 5), (6, 9, 5)) 204
((6, 3, 6), (6, 5, 6)) 240
((2, 2, 6), (2, 4, 6)) 242
((1, 0, 7), (4, 0, 7)) 253
((4, 7, 11), (4, 7, 11)) 259
((6, 9, 8), (6, 9, 10)) 302
((6, 8, 8), (9, 8, 8)) 305
((6, 4, 9), (6, 5, 9)) 333
((6, 2, 9), (6, 3, 9)) 336
((2, 4, 8), (2, 5, 8)) 337
((3, 0, 8), (3, 2

7474

In [670]:
for k,v in input_height_map.items():
    if v != []:
        print(k)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
152
153
154
155


In [680]:
input_height_map[148]

[((7, 0, 289), (7, 2, 289)),
 ((4, 2, 291), (4, 2, 294)),
 ((8, 6, 292), (8, 7, 292)),
 ((6, 5, 293), (6, 5, 296)),
 ((2, 5, 293), (5, 5, 293)),
 ((5, 4, 294), (8, 4, 294))]

In [677]:
input_height_map[149]

[((6, 2, 290), (9, 2, 290)),
 ((8, 7, 294), (8, 7, 296)),
 ((2, 5, 294), (2, 5, 296)),
 ((5, 1, 296), (5, 4, 296)),
 ((7, 4, 296), (8, 4, 296))]

In [676]:
input_height_map[150]

[((6, 0, 293), (6, 2, 293))]

In [675]:
input_height_map[151]

[]

In [674]:
input_height_map[152]

[((3, 2, 295), (4, 2, 295)), ((4, 5, 298), (6, 5, 298))]

In [673]:
input_height_map[153]

[((4, 1, 297), (4, 3, 297)),
 ((5, 5, 299), (5, 7, 299)),
 ((4, 4, 300), (4, 7, 300))]

In [661]:
input_height_map[154]

[((4, 1, 300), (5, 1, 300)),
 ((4, 5, 302), (4, 9, 302)),
 ((2, 4, 302), (5, 4, 302))]

In [662]:
input_height_map[155]

[((5, 1, 301), (6, 1, 301)),
 ((3, 4, 303), (3, 6, 303)),
 ((2, 4, 304), (2, 4, 305)),
 ((4, 4, 305), (5, 4, 305))]

In [681]:
test_height_map_2 = {
    [i for i in range(8)][j]:input_height_map[j+148] for j in range(7)
}

In [682]:
test_height_map_2

{0: [((7, 0, 289), (7, 2, 289)),
  ((4, 2, 291), (4, 2, 294)),
  ((8, 6, 292), (8, 7, 292)),
  ((6, 5, 293), (6, 5, 296)),
  ((2, 5, 293), (5, 5, 293)),
  ((5, 4, 294), (8, 4, 294))],
 1: [((6, 2, 290), (9, 2, 290)),
  ((8, 7, 294), (8, 7, 296)),
  ((2, 5, 294), (2, 5, 296)),
  ((5, 1, 296), (5, 4, 296)),
  ((7, 4, 296), (8, 4, 296))],
 2: [((6, 0, 293), (6, 2, 293))],
 3: [],
 4: [((3, 2, 295), (4, 2, 295)), ((4, 5, 298), (6, 5, 298))],
 5: [((4, 1, 297), (4, 3, 297)),
  ((5, 5, 299), (5, 7, 299)),
  ((4, 4, 300), (4, 7, 300))],
 6: [((4, 1, 300), (5, 1, 300)),
  ((4, 5, 302), (4, 9, 302)),
  ((2, 4, 302), (5, 4, 302))]}

In [683]:
part_2(test_height_map_2)

((7, 0, 289), (7, 2, 289)) 2
((4, 2, 291), (4, 2, 294)) 5
((8, 6, 292), (8, 7, 292)) 6
((6, 5, 293), (6, 5, 296)) 10
((2, 5, 293), (5, 5, 293)) 11
((5, 4, 294), (8, 4, 294)) 12
((6, 2, 290), (9, 2, 290)) 13
((8, 7, 294), (8, 7, 296)) 13
((2, 5, 294), (2, 5, 296)) 13
((5, 1, 296), (5, 4, 296)) 13
((7, 4, 296), (8, 4, 296)) 13
((6, 0, 293), (6, 2, 293)) 13
((3, 2, 295), (4, 2, 295)) 15
((4, 5, 298), (6, 5, 298)) 18
((4, 1, 297), (4, 3, 297)) 19
((5, 5, 299), (5, 7, 299)) 19
((4, 4, 300), (4, 7, 300)) 20
((4, 1, 300), (5, 1, 300)) 20
((4, 5, 302), (4, 9, 302)) 20
((2, 4, 302), (5, 4, 302)) 20


20

In [656]:
part_2(input_height_map)

((4, 0, 1), (5, 0, 1)) 0
((1, 9, 1), (3, 9, 1)) 1
((4, 1, 1), (6, 1, 1)) 1
((3, 1, 1), (3, 1, 3)) 1
((2, 6, 1), (2, 6, 3)) 5
((6, 4, 1), (6, 4, 3)) 66
((9, 6, 1), (9, 6, 2)) 66
((8, 2, 1), (8, 4, 1)) 66
((9, 2, 1), (9, 3, 1)) 66
((7, 6, 1), (7, 8, 1)) 68
((9, 4, 2), (9, 4, 3)) 70
((0, 5, 2), (2, 5, 2)) 72
((2, 0, 2), (2, 0, 5)) 84
((1, 3, 2), (1, 3, 5)) 85
((7, 2, 2), (7, 4, 2)) 85
((7, 1, 2), (9, 1, 2)) 86
((4, 2, 2), (4, 4, 2)) 89
((0, 8, 2), (2, 8, 2)) 91
((9, 0, 3), (9, 0, 4)) 91
((6, 8, 3), (6, 9, 3)) 92
((2, 4, 3), (3, 4, 3)) 94
((8, 9, 3), (9, 9, 3)) 95
((1, 6, 3), (1, 6, 4)) 95
((1, 9, 3), (1, 9, 4)) 95
((7, 5, 3), (7, 7, 3)) 96
((5, 1, 3), (7, 1, 3)) 98
((2, 3, 3), (5, 3, 3)) 99
((3, 2, 4), (5, 2, 4)) 100
((0, 8, 4), (3, 8, 4)) 101
((0, 5, 5), (0, 7, 5)) 102
((9, 1, 5), (9, 1, 5)) 102
((5, 8, 4), (6, 8, 4)) 102
((8, 7, 5), (8, 9, 5)) 102
((3, 4, 5), (3, 7, 5)) 103
((9, 3, 5), (9, 4, 5)) 104
((7, 6, 6), (8, 6, 6)) 104
((5, 3, 4), (5, 5, 4)) 104
((6, 1, 6), (7, 1, 6)) 105
((3, 2

7474

In [ ]:
# 10816 is too low